# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [2]:
from dotenv import load_dotenv
load_dotenv(r'C:\Users\DELL\OneDrive\Desktop\chatbot\env')  # Load environment variables, such as API keys


## LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Initialize the LLM
llm = ChatOpenAI(temperature=0.9, model='gpt-3.5-turbo')

# Define a prompt template
prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")

# Create an LLMChain with the prompt
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain with a specific product input
product = "Queen Size Sheet Set"
print(chain.run(product))  # Output will be the suggested company name


"Royal Slumber Co."


In [3]:
## Sequential Chains

## a. SimpleSequentialChain
from langchain.chains import SimpleSequentialChain

# Define the first prompt template
first_prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")

# Define the second prompt template
second_prompt = ChatPromptTemplate.from_template("Write a 20-word description for the following company: {company_name}")

# Create two separate LLMChains
chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Combine the chains in sequence
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

# Run the combined chain with a product input
print(overall_simple_chain.run(product))

## b.SequentialChain
import pandas as pd
from langchain.chains import SequentialChain

# Load data for input
df = pd.DataFrame({"Review": ["This product is excellent and affordable!"]})

# Define each prompt in the chain
first_prompt = ChatPromptTemplate.from_template("Translate the following review to English:\n\n{Review}")
second_prompt = ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence:\n\n{English_Review}")
third_prompt = ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}")
fourth_prompt = ChatPromptTemplate.from_template("Write a follow-up response to the following summary in the specified language:\n\nSummary: {summary}\n\nLanguage: {language}")

# Create each chain step with its prompt and set output keys
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

# Combine chains with specified input/output mappings
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

# Run the chain with a sample review
review = df.Review[0]
print(overall_chain(review))




> Entering new SimpleSequentialChain chain...
"Royal Dreams Linens"
Luxurious and opulent linens fit for royalty, offering a range of high-quality bedding sets, duvet covers, and pillowcases in stylish designs.

> Finished chain.
Luxurious and opulent linens fit for royalty, offering a range of high-quality bedding sets, duvet covers, and pillowcases in stylish designs.


C:\Users\DELL\AppData\Local\Temp\ipykernel_32272\1677902372.py:51: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(overall_chain(review))




> Entering new SequentialChain chain...

> Finished chain.
{'Review': 'This product is excellent and affordable!', 'English_Review': 'Este producto es excelente y asequible!', 'summary': 'This product is excellent and affordable.', 'followup_message': "Thank you for your positive feedback on our product! We are thrilled to hear that you found it excellent and affordable. We strive to provide high-quality products at affordable prices, so it's great to know that we are meeting our goal. If you have any more feedback or suggestions, please feel free to share. We appreciate your support!"}


In [10]:
## Router Chain: Directs Input to Different Chains Based on the Topic


from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

# Define specialized prompt templates for different subjects
physics_template = "You are a physics professor. Answer the question concisely:\n{input}"
math_template = "You are a mathematician. Break down the problem and answer:\n{input}"
history_template = "You are a historian. Provide context and answer:\n{input}"
cs_template = "You are a computer scientist. Explain step-by-step:\n{input}"

# Set up prompt info for each specialized chain
prompt_infos = [
    {"name": "physics", "description": "Good for physics questions", "prompt_template": physics_template},
    {"name": "math", "description": "Good for math questions", "prompt_template": math_template},
    {"name": "history", "description": "Good for history questions", "prompt_template": history_template},
    {"name": "computer science", "description": "Good for computer science questions", "prompt_template": cs_template}
]

# Initialize the LLM
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')

# Create the destination chains based on the prompt info
destination_chains = {
    info["name"]: LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(info["prompt_template"]))
    for info in prompt_infos
}

# Define a default chain to handle cases where the input doesn't fit other categories
default_template = "Please answer this general question concisely:\n{input}"
default_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(default_template))


# Create a string of all the available destinations for the routing template
destinations_str = "\n".join([f"{p['name']}: {p['description']}" for p in prompt_infos])

# Define the router prompt template with JSON response formatting
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

# Create prompt and router
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(template=router_template, input_variables=["input"], output_parser=RouterOutputParser())
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# MultiPromptChain directs questions to the best chain based on the topic
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

# Test Router Chain
print(chain.run("What is black body radiation?"))  # Should go to physics chain
print(chain.run("What is 2 + 2"))  # Should go to math chain
print(chain.run("Why does every cell in our body contain DNA?"))  # Uses default chain if topic doesn’t fit others



<>:40: SyntaxWarning: invalid escape sequence '\ '
<>:40: SyntaxWarning: invalid escape sequence '\ '
C:\Users\DELL\AppData\Local\Temp\ipykernel_32272\76802467.py:40: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body, at a given temperature.


> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2'}
> Finished chain.
The problem is asking for the sum of 2 and 2. To solve this, we simply add the two numbers together:

2 + 2 = 4

Therefore, the answer is 4.


> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

C:\Users\DELL\AppData\Local\Temp\ipykernel_32272\76802467.py:40: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


ValueError: Received invalid destination chain name 'biology'